In [1]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1
!pip install git+https://github.com/tensorflow/docs > /dev/null 2>&1
!pip install gym[classic_control]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.5/821.5 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 45.7 MB/s eta 0:00:00
  Attempting uninstall: pygame
    Found existing installation: pygame 2.5.2
    Uninstalling pygame-2.5.2:
      Successfully uninstalled pygame-2.5.2


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import namedtuple, deque
import torch.optim as optim
import datetime
import gym
from gym.wrappers.record_video import RecordVideo
import glob
import io
import base64
import matplotlib.pyplot as plt
from IPython.display import HTML
from pyvirtualdisplay import Display
import tensorflow as tf
from IPython import display as ipythondisplay
from PIL import Image
import tensorflow_probability as tfp

/usr/local/lib/python3.10/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


In [3]:
'''
Bunch of Hyper parameters (Which you might have to tune later)
'''
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
LR = 5e-5             # learning rate

env = gym.make('Acrobot-v1')
state = env.reset()
print(state)

[ 0.9969729  -0.0777493   0.9962257   0.08680018  0.09011854  0.0894108 ]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
class QNetwork1(nn.Module):

  def __init__(self, state_size, action_size, hl_size):
    super(QNetwork1,self).__init__()
    self.input_layer = nn.Linear(state_size,hl_size)
    self.value_layer = nn.Linear(hl_size,1)
    self.advantage_layer = nn.Linear(hl_size,action_size)

  def forward(self,state):
    x = F.relu(self.input_layer(state))
    value = self.value_layer(x)
    advantage = self.advantage_layer(x)
    return value , advantage


network = QNetwork1(6,3,2)
print(network.forward(torch.tensor([[1,1,1,1,1,4],[1,2,3,4,5,4]],dtype=torch.float32)))

(tensor([[ 0.1462],
        [-0.1963]], grad_fn=<AddmmBackward0>), tensor([[ 0.4298, -0.0550, -0.5994],
        [ 1.2368,  0.3832, -1.6038]], grad_fn=<AddmmBackward0>))


## Replay Buffer

In [5]:
import random
import torch
import numpy as np
from collections import deque, namedtuple

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)

    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)

    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

#DQN

##mean(A)

In [ ]:
class TutorialAgent():

    def __init__(self, state_size, action_size, seed, hl_size = 64, batch_size = 32, lr = 1e-5):

        ''' Agent Environment Interaction '''
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)
        self.batch_size = batch_size
        ''' Q-Network '''
        self.qnetwork_local = QNetwork1(state_size, action_size, hl_size = hl_size).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=lr)

        ''' Replay memory '''
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, batch_size, seed)

        ''' Initialize time step (for updating every UPDATE_EVERY steps)           -Needed for Q Targets '''
        self.t_step = 0

    def step(self, state, action, reward, next_state, done):

        ''' Save experience in replay memory '''
        self.memory.add(state, action, reward, next_state, done)

        ''' If enough samples are available in memory, get random subset and learn '''
        if len(self.memory) >= self.batch_size:
            experiences = self.memory.sample()
            self.learn(experiences, GAMMA)


    def act(self, state, eps=0.5):

        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            _,action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        ''' Epsilon-greedy action selection (Already Present) '''
        if random.random() > eps:
            return torch.argmax(action_values.to('cpu'),dim=1).item()
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences, gamma):
        """ +E EXPERIENCE REPLAY PRESENT """
        states, actions, rewards, next_states, dones = experiences
        ''' Get max predicted Q values (for next states) from target model'''
        V_targets_next, A_targets_next = self.qnetwork_local(next_states)
        Q_targets_next = torch.add(V_targets_next, A_targets_next - A_targets_next.mean(dim=1, keepdim=True)).detach().max(1)[0].unsqueeze(1)

        ''' Compute Q targets for current states '''
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        ''' Get expected Q values from local model '''
        V_targets, A_targets = self.qnetwork_local(states)
        Q_expected = torch.add(V_targets, A_targets - A_targets.mean(dim=1, keepdim=True)).gather(1, actions)

        ''' Compute loss '''
        loss = F.mse_loss(Q_expected, Q_targets)

        ''' Minimize the loss '''
        self.optimizer.zero_grad()
        loss.backward()
        ''' Gradiant Clipping '''
        """ +T TRUNCATION PRESENT """
        for param in self.qnetwork_local.parameters():
            param.grad.data.clamp_(-1, 1)

        self.optimizer.step()




In [ ]:
''' Defining DQN Algorithm '''

state_shape = env.observation_space.shape[0]
action_shape = env.action_space.n


def dqn(n_episodes=500, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995,print_reward = False):

    scores_window = deque(maxlen=100)
    ''' last 100 scores for checking if the avg is more than 195 '''

    eps = eps_start
    ''' initialize epsilon '''
    total_reward = []
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break

        scores_window.append(score)
        total_reward.append(score)
        eps = max(eps_end, eps_decay*eps)
        ''' decrease epsilon '''
        # print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, score), end="")
        if i_episode % 10 == 0 and print_reward == True:
           print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=-110.0:
           print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
           break
    return total_reward


##Batch size 32

In [ ]:
BATCH_SIZE = [32]
LR = [5e-4,1e-4,5e-5]
HL_size = [8,16,32]

In [ ]:
reward_max = []
i = 0
for batch_size in BATCH_SIZE:
  for lr in LR:
    for hl_size in HL_size:
      agent = TutorialAgent(state_size=state_shape,action_size = action_shape,seed = 0,hl_size = hl_size , batch_size = batch_size , lr = lr)
      reward_max.append(dqn())
      i+=1
      print(f'Copmleted iteration {i}')
min_size = min([len(i) for i in reward_max])
reward_max = np.array([i[:min_size] for i in reward_max]).reshape(len(LR),len(HL_size),-1)
np.save('HP_32_mean.npy',reward_max)

Copmleted iteration 1
Copmleted iteration 2
Copmleted iteration 3

Environment solved in 326 episodes!	Average Score: -109.83
Copmleted iteration 4
Copmleted iteration 5
Copmleted iteration 6
Copmleted iteration 7
Copmleted iteration 8
Copmleted iteration 9


##Batch size 64

In [ ]:
BATCH_SIZE = [64]
LR = [5e-4,1e-4,5e-5]
HL_size = [8,16,32]

In [ ]:
reward_max = []
i = 0
for batch_size in BATCH_SIZE:
  for lr in LR:
    for hl_size in HL_size:
      agent = TutorialAgent(state_size=state_shape,action_size = action_shape,seed = 0,hl_size = hl_size , batch_size = batch_size , lr = lr)
      reward_max.append(dqn())
      i+=1
      print(f'Copmleted iteration {i}')
min_size = min([len(i) for i in reward_max])
reward_max = np.array([i[:min_size] for i in reward_max]).reshape(len(LR),len(HL_size),-1)
np.save('HP_64_mean.npy',reward_max)

Copmleted iteration 1
Copmleted iteration 2
Copmleted iteration 3

Environment solved in 358 episodes!	Average Score: -109.07
Copmleted iteration 4
Copmleted iteration 5

Environment solved in 431 episodes!	Average Score: -109.90
Copmleted iteration 6

Environment solved in 450 episodes!	Average Score: -109.93
Copmleted iteration 7

Environment solved in 308 episodes!	Average Score: -109.73
Copmleted iteration 8

Environment solved in 369 episodes!	Average Score: -109.86
Copmleted iteration 9


##Batch size 128

In [ ]:
BATCH_SIZE = [128]
LR = [5e-4,1e-4,5e-5]
HL_size = [8,16,32]

In [ ]:
reward_max = []
i = 0
for batch_size in BATCH_SIZE:
  for lr in LR:
    for hl_size in HL_size:
      agent = TutorialAgent(state_size=state_shape,action_size = action_shape,seed = 0,hl_size = hl_size , batch_size = batch_size , lr = lr)
      reward_max.append(dqn())
      i+=1
      print(f'Copmleted iteration {i}')
min_size = min([len(i) for i in reward_max])
reward_max = np.array([i[:min_size] for i in reward_max]).reshape(len(LR),len(HL_size),-1)
np.save('HP_128_mean.npy',reward_max)

Copmleted iteration 1

Environment solved in 450 episodes!	Average Score: -109.30
Copmleted iteration 2
Copmleted iteration 3
Copmleted iteration 4

Environment solved in 351 episodes!	Average Score: -109.79
Copmleted iteration 5

Environment solved in 358 episodes!	Average Score: -109.88
Copmleted iteration 6

Environment solved in 321 episodes!	Average Score: -109.95
Copmleted iteration 7
Copmleted iteration 8
Copmleted iteration 9


##max(A)

In [6]:
class TutorialAgent():

    def __init__(self, state_size, action_size, seed, hl_size = 64, batch_size = 32, lr = 1e-5):

        ''' Agent Environment Interaction '''
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)
        self.lr = lr
        self.batch_size = batch_size
        ''' Q-Network '''
        self.qnetwork_local = QNetwork1(state_size, action_size, hl_size = hl_size).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=self.lr)

        ''' Replay memory '''
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, batch_size, seed)


    def step(self, state, action, reward, next_state, done):

        ''' Save experience in replay memory '''
        self.memory.add(state, action, reward, next_state, done)

        ''' If enough samples are available in memory, get random subset and learn '''
        if len(self.memory) >= self.batch_size:
            experiences = self.memory.sample()
            self.learn(experiences, GAMMA)


    def act(self, state, eps=0.5):

        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            _,action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        ''' Epsilon-greedy action selection (Already Present) '''
        if random.random() > eps:
            return torch.argmax(action_values,dim=1).item()
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences, gamma):
        """ +E EXPERIENCE REPLAY PRESENT """
        states, actions, rewards, next_states, dones = experiences
        ''' Get max predicted Q values (for next states) from target model'''
        V_targets_next, A_targets_next = self.qnetwork_local(next_states)
        Q_targets_next = torch.add(V_targets_next, A_targets_next - torch.max(A_targets_next, dim=1).values.unsqueeze(1)).detach().max(1)[0].unsqueeze(1)

        ''' Compute Q targets for current states '''
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        ''' Get expected Q values from local model '''
        V_targets, A_targets = self.qnetwork_local(states)
        Q_expected = torch.add(V_targets, A_targets - torch.max(A_targets, dim=1).values.unsqueeze(1)).gather(1, actions)

        ''' Compute loss '''
        loss = F.mse_loss(Q_expected, Q_targets)

        ''' Minimize the loss '''
        self.optimizer.zero_grad()
        loss.backward()
        ''' Gradiant Clipping '''
        """ +T TRUNCATION PRESENT """
        for param in self.qnetwork_local.parameters():
            param.grad.data.clamp_(-1, 1)

        self.optimizer.step()




In [7]:
''' Defining DQN Algorithm '''

state_shape = env.observation_space.shape[0]
action_shape = env.action_space.n


def dqn(n_episodes=500, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995,print_reward = False):

    scores_window = deque(maxlen=100)
    ''' last 100 scores for checking if the avg is more than 195 '''
    total_reward = []
    eps = eps_start
    ''' initialize epsilon '''

    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break
        total_reward.append(score)
        scores_window.append(score)

        eps = max(eps_end, eps_decay*eps)
        ''' decrease epsilon '''

        if i_episode % 10 == 0 and print_reward == True:
           print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=-110:
           print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
           break
    return total_reward

''' Trial run to check if algorithm runs and saves the data '''

# begin_time = datetime.datetime.now()

# agent = TutorialAgent(state_size=state_shape,action_size = action_shape,seed = 0)
# dqn(print_reward = True)

# time_taken = datetime.datetime.now() - begin_time

# print(time_taken)

' Trial run to check if algorithm runs and saves the data '

##Batch size 32

In [ ]:
BATCH_SIZE = [32]
LR = [5e-4,1e-4,5e-5]
HL_size = [8,16,32]

In [ ]:
reward_max = []
i = 0
for batch_size in BATCH_SIZE:
  for lr in LR:
    for hl_size in HL_size:
      agent = TutorialAgent(state_size=state_shape,action_size = action_shape,seed = 0,hl_size = hl_size , batch_size = batch_size , lr = lr)
      reward_max.append(dqn())
      i+=1
      print(f'Copmleted iteration {i}')
min_size = min([len(i) for i in reward_max])
reward_max = np.array([i[:min_size] for i in reward_max]).reshape(len(LR),len(HL_size),-1)
np.save('HP_32_max.npy',reward_max)

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Copmleted iteration 1
Copmleted iteration 2
Copmleted iteration 3
Copmleted iteration 4
Copmleted iteration 5
Copmleted iteration 6
Copmleted iteration 7
Copmleted iteration 8
Copmleted iteration 9


##Batch size 64

In [ ]:
BATCH_SIZE = [64]
LR = [5e-4,1e-4,5e-5]
HL_size = [8,16,32]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
reward_max = []
i = 0
for batch_size in BATCH_SIZE:
  for lr in LR:
    for hl_size in HL_size:
      agent = TutorialAgent(state_size=state_shape,action_size = action_shape,seed = 0,hl_size = hl_size , batch_size = batch_size , lr = lr)
      reward_max.append(dqn())
      i+=1
      print(f'Copmleted iteration {i}')
min_size = min([len(i) for i in reward_max])
reward_max = np.array([i[:min_size] for i in reward_max]).reshape(len(LR),len(HL_size),-1)
np.save('HP_64_max.npy',reward_max)

Copmleted iteration 1
Copmleted iteration 2
Copmleted iteration 3
Copmleted iteration 4
Copmleted iteration 5
Copmleted iteration 6
Copmleted iteration 7
Copmleted iteration 8
Copmleted iteration 9


##Batch size 128

In [ ]:
BATCH_SIZE = [128]
LR = [5e-4,1e-4,5e-5]
HL_size = [8,16,32]

In [ ]:
reward_max = []
i = 0
for batch_size in BATCH_SIZE:
  for lr in LR:
    for hl_size in HL_size:
      agent = TutorialAgent(state_size=state_shape,action_size = action_shape,seed = 0,hl_size = hl_size , batch_size = batch_size , lr = lr)
      reward_max.append(dqn())
      i+=1
      print(f'Copmleted iteration {i}')
min_size = min([len(i) for i in reward_max])
reward_max = np.array([i[:min_size] for i in reward_max]).reshape(len(LR),len(HL_size),-1)
np.save('HP_128_max.npy',reward_max)

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Copmleted iteration 1
Copmleted iteration 2
Copmleted iteration 3

Environment solved in 441 episodes!	Average Score: -109.65
Copmleted iteration 4

Environment solved in 471 episodes!	Average Score: -109.44
Copmleted iteration 5
Copmleted iteration 6

Environment solved in 454 episodes!	Average Score: -109.53
Copmleted iteration 7
Copmleted iteration 8
Copmleted iteration 9
